In [1]:
import numpy as np
import time


index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from scipy.sparse import vstack

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out, indices_out = [], [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]
        indices_c = np.where(y == label)[0]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        # Ensure at least one sample per class
        n_sample = max(1, n_sample)

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])
        indices_out.append(indices_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0), np.concatenate(indices_out, axis=0)

def batch_mode_active_learning(X_pool, y_pool, model, fraction=0.2, random_state=42, max_clusters_per_iter=1000, max_top_k=10000, batch_size=10000):
    selected_indices = []
    rng = np.random.RandomState(seed=random_state)
    n_total = len(X_pool)
    n_select = max(1, int(n_total * fraction))  # 目标选择样本数
    classes = np.unique(y_pool)

    # Convert sparse to dense if necessary (handle in batches if needed)
    is_sparse = hasattr(X_pool, 'toarray')

    # Step 1: Ensure at least one sample per class directly from X_pool
    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        if len(idx_c) == 0:
            continue
        # Randomly select one sample for this class
        selected_idx = rng.choice(idx_c, size=1)[0]
        selected_indices.append(selected_idx)

    # Step 2: Compute uncertainties for X_pool in batches
    uncertainties = np.zeros(n_total)
    for start in range(0, n_total, batch_size):
        end = min(start + batch_size, n_total)
        X_batch = X_pool[start:end].toarray() if is_sparse else X_pool[start:end]
        probs = model.predict_proba(X_batch)
        uncertainties[start:end] = 1 - np.max(probs, axis=1)

    # Step 3: Iterative clustering to select remaining samples
    n_remaining = n_select - len(selected_indices)
    current_indices = np.setdiff1d(np.arange(n_total), selected_indices)  # 剩余样本索引

    while n_remaining > 0 and len(current_indices) > 0:
        # Determine clusters for this iteration
        n_clusters = min(max_clusters_per_iter, n_remaining, len(current_indices))
        if n_clusters == 0:
            break

        # Select top-k uncertain instances
        top_k = min(n_clusters * 5, len(current_indices) - 1, max_top_k)
        if top_k <= 0:
            break

        current_uncertainties = uncertainties[current_indices]
        top_indices = np.argpartition(-current_uncertainties, top_k)[:top_k]
        top_global_indices = current_indices[top_indices]

        # Prepare data for clustering
        X_top = X_pool[top_global_indices].toarray() if is_sparse else X_pool[top_global_indices]

        # Adjust n_clusters to be at most the number of samples
        n_clusters = min(n_clusters, len(top_indices))
        if n_clusters == 0:
            break

        # Cluster with MiniBatchKMeans
        kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=random_state, batch_size=1000, max_iter=100)
        kmeans.fit(X_top)

        # Vectorized distance calculation
        distances = np.linalg.norm(X_top[:, np.newaxis] - kmeans.cluster_centers_, axis=2)
        cluster_closest = np.argmin(distances, axis=0)
        selected_indices.extend(top_global_indices[cluster_closest])

        # Update remaining samples and uncertainties
        current_indices = np.setdiff1d(current_indices, top_global_indices[cluster_closest])
        n_remaining -= n_clusters

    # Step 4: Finalize outputs
    selected_indices = np.array(selected_indices[:n_select])  # Trim if oversampled
    X_selected = X_pool[selected_indices]
    y_selected = y_pool[selected_indices]
    remaining_idx = np.setdiff1d(np.arange(n_total), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    # Step 5: Verify each class appears at least once
    selected_class_counts = np.bincount(y_selected, minlength=len(classes))
    missing_classes = classes[selected_class_counts == 0]
    if len(missing_classes) > 0:
        print(f"Warning: Missing classes {missing_classes}. Adding one sample per missing class.")
        for c in missing_classes:
            idx_c = np.where(y_pool == c)[0]
            if len(idx_c) > 0:
                selected_idx = rng.choice(idx_c, size=1)[0]
                selected_indices = np.append(selected_indices, selected_idx)
                n_select += 1  # Adjust n_select to account for extra samples
        # Update outputs after adding missing classes
        selected_indices = selected_indices[:n_select]
        X_selected = X_pool[selected_indices]
        y_selected = y_pool[selected_indices]
        remaining_idx = np.setdiff1d(np.arange(n_total), selected_indices)
        X_rest = X_pool[remaining_idx]
        y_rest = y_pool[remaining_idx]

    # Print selected samples' label distribution
    print(f"Selected samples (fraction={fraction}): {len(y_selected)}")
    print(f"Label distribution: {np.bincount(y_selected, minlength=len(classes))}")

    return X_rest, X_selected, y_rest, y_selected

X_pool = x_test_adv_0_1 
y_pool = y_label_0_1
X_train_init, y_train_init, _ = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)

# Perform batch mode active learning with fractions 50%, 20%, 1%
_, X_50, _, y_50 = batch_mode_active_learning(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.5, random_state=42
)

Selected samples (fraction=0.5): 512869
Label distribution: [239952    823   4103   7501   1493  15445    448   1655 240119    944
    386]


In [24]:
X_50.shape

(512869, 56)

In [25]:
# from xgboost import XGBClassifier

# print('xgb')
# xgb50 = XGBClassifier()
# xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt50 = DecisionTreeClassifier()
# dt50.fit(X_50, y_50)

xgb
RF
DT


DecisionTreeClassifier()

In [ ]:
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [26]:
import numpy as np



base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [27904   184  2252   918    62  2623     1   219 79535   201    72]
(27904, 56) (27904,)
Save 0 to baseline_Def1.npy
(184, 56) (184,)
Save 1 to baseline_Def2.npy
(2252, 56) (2252,)
Save 2 to baseline_Def3.npy
(918, 56) (918,)
Save 3 to baseline_Def4.npy
(62, 56) (62,)
Save 4 to baseline_Def5.npy
(2623, 56) (2623,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(219, 56) (219,)
Save 7 to baseline_Def8.npy
(79535, 56) (79535,)
Save 8 to baseline_Def9.npy
(201, 56) (201,)
Save 9 to baseline_Def10.npy
(72, 56) (72,)
Save 10 to baseline_Def11.npy
Execution Time: 0.310912 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [128019    302   1719   8620    647  97602    419 104365    178     42]
(128019, 56) (128019,)
Save 0 to BIM_Def1.npy
(302, 56) (302,)
Save 1 to BIM_Def2.npy
(1719, 56) (1719,)
Save 2 to BIM_Def3.npy
(8620, 56) (8620,)
Save 3 to BIM_Def4.npy
(647, 56) (647,)
Save 4 to BIM_Def5.npy
(97602, 56) (97602,)
Save 5 

In [27]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [33117   251  2028   991   105  3261   276 73642   211    89]
(33117, 56) (33117,)
Save 0 to baseline_Def1.npy
(251, 56) (251,)
Save 1 to baseline_Def2.npy
(2028, 56) (2028,)
Save 2 to baseline_Def3.npy
(991, 56) (991,)
Save 3 to baseline_Def4.npy
(105, 56) (105,)
Save 4 to baseline_Def5.npy
(3261, 56) (3261,)
Save 5 to baseline_Def6.npy
(276, 56) (276,)
Save 7 to baseline_Def8.npy
(73642, 56) (73642,)
Save 8 to baseline_Def9.npy
(211, 56) (211,)
Save 9 to baseline_Def10.npy
(89, 56) (89,)
Save 10 to baseline_Def11.npy
Execution Time: 3.071802 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [141744      5   1336   8284    120 100291     34  89836      6    257]
(141744, 56) (141744,)
Save 0 to BIM_Def1.npy
(5, 56) (5,)
Save 1 to BIM_Def2.npy
(1336, 56) (1336,)
Save 2 to BIM_Def3.npy
(8284, 56) (8284,)
Save 3 to BIM_Def4.npy
(120, 56) (120,)
Save 4 to BIM_Def5.npy
(100291, 56) (100291,)
Save 5 to BIM_Def6.npy
(34, 56) (34,)
Save 7 to BIM_Def8.

In [28]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [49571   483  2387  8074   337  7316     1   304 44944   417   137]
(49571, 56) (49571,)
Save 0 to baseline_Def1.npy
(483, 56) (483,)
Save 1 to baseline_Def2.npy
(2387, 56) (2387,)
Save 2 to baseline_Def3.npy
(8074, 56) (8074,)
Save 3 to baseline_Def4.npy
(337, 56) (337,)
Save 4 to baseline_Def5.npy
(7316, 56) (7316,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(304, 56) (304,)
Save 7 to baseline_Def8.npy
(44944, 56) (44944,)
Save 8 to baseline_Def9.npy
(417, 56) (417,)
Save 9 to baseline_Def10.npy
(137, 56) (137,)
Save 10 to baseline_Def11.npy
Execution Time: 0.098470 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [110764   1404   7890  29613   5664 129434      9   2006  52800   1447
    882]
(110764, 56) (110764,)
Save 0 to BIM_Def1.npy
(1404, 56) (1404,)
Save 1 to BIM_Def2.npy
(7890, 56) (7890,)
Save 2 to BIM_Def3.npy
(29613, 56) (29613,)
Save 3 to BIM_Def4.npy
(5664, 56) (5664,)
Save 4 to BIM_Def5.npy
(1294

In [29]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ActiveLearning_BatchMode/UNSW_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [30]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_ActiveLearning_BatchMode.csv")

In [31]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB50,27904,184,2252,918,62,2623,1,219,79535,201,72,113971
1,XGB50,128019,302,1719,8620,647,97602,0,419,104365,178,42,341913
2,XGB50,128322,54,1597,8590,154,98016,3,7575,97486,82,34,341913
3,XGB50,128019,302,1719,8620,647,97602,0,419,104365,178,42,341913
4,XGB50,119892,877,2982,232,357,138369,33632,42749,2398,349,76,341913
5,XGB50,194115,623,4909,4298,729,99738,0,834,36031,414,222,341913
6,XGB50,79329,524,7227,3138,180,7596,3,657,242473,570,216,341913
7,XGB50,128057,84,602,5734,147,119372,1,90,81171,4792,1863,341913
8,XGB50,149590,1037,1056,4125,891,81769,0,107,103244,87,7,341913
9,XGB50,144876,113,1902,5891,1297,87437,0,360,99887,129,21,341913
